In [17]:
def load():
    global torch, np, resnext, Compose, Scale, CenterCrop, ToTensor, Normalize, TemporalCenterCrop, spatial_transforms, \
        target_transforms, temporal_transforms, PILIm, ImageOps, ImShow, plt, cv2, os, glob, csv, \
        size, norm_value, duration, classes
    
    import torch
    import numpy as np
    import resnext
    from spatial_transforms import Compose, Scale, CenterCrop, ToTensor, Normalize
    from temporal_transforms import TemporalCenterCrop
    import spatial_transforms
    import target_transforms
    import temporal_transforms
    from PIL import Image as PILIm
    from PIL import ImageOps
    from IPython.display import Image as ImShow
    import matplotlib.pyplot as plt
    #import torchvision
    import cv2
    import os
    import glob
    import csv

    size = 112
    norm_value = 1
    duration = 32
    classes = 27

In [3]:
#LOAD MODEL
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath, map_location='cpu')

    my_dict = checkpoint['state_dict']
    my_dict_copy = my_dict.copy()

    for key in my_dict_copy.keys():
        newkey = key[7:]
        my_dict[newkey] = my_dict.pop(key)

    checkpoint['state_dict'] = my_dict 
    
    
    
    model = resnext.resnet101(
        num_classes=classes,
        shortcut_type='B',
        cardinality=32,
        sample_size=size,
        sample_duration=duration)

    model.load_state_dict(checkpoint['state_dict'])

    model.eval()
    return model

In [4]:
def get_mean(normval=255):
    return [114.7748 / normval, 107.7354 / normval, 99.4750 / normval ]

In [5]:
#DEFINE THE TRANSFORMS
def getTransforms():
    scales = [1]
    for i in range(1, 5):
        scales.append(scales[-1] * 0.84089641525)
        
    norm_method = Normalize(get_mean(norm_value), [1, 1, 1])
            
    spatial_transform = spatial_transforms.Compose([
        Scale(size),
        CenterCrop(size),
        ToTensor(norm_value), norm_method 
    ])

    temporal_transform = spatial_transforms.Compose([
        TemporalCenterCrop(duration) 
    ])
    return spatial_transform, temporal_transform

In [6]:
def ClearDir(dirname, filetype='jpg'):
    import glob, os, os.path

    filelist = glob.glob(os.path.join(dirname, "*."+filetype))
    for f in filelist:
        os.remove(f)

In [6]:
def PlayVidInput():
    import cv2
    import numpy as np
    import os
    import random
    from os.path import isfile, join
    pathIn= 'vidinput/'
    #print(pathIn)
    pathOut = './gesturevideos/video' + str(random.randint(0,1000)) + '.avi'
    fps = 15
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: x[5:-4])
    files.sort()
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: x[5:-4])
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
    
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [9]:
#TURN THE 37 FRAMES FROM ./TESTVID INTO TENSOR INPUT <1,3,32,112,112>
def TransformData(spatial_transform, temporal_transform, framesTT, Show=False):
    a=[]
    for frame in framesTT:
        image = PILIm.fromarray(frame.astype('uint8'), mode='RGB').convert("RGB")
        image.save("YOLO.png")
        a.append(image)

    input_temp = temporal_transform(a)
    
    if Show:
        ClearDir('vidinput/','png')
        count = 0
        for image in input_temp:
            image = np.array(image)
            if count<10:
                #image.save("vidinput/frame0%d.png" % count)
                cv2.imwrite("vidinput/frame0%d.png" % count, image)

            else:
                cv2.imwrite("vidinput/frame%d.png" % count, image)
            count +=1
        PlayVidInput()
    
    #for image in input_temp:
    #    image.save("noncroppedimage.png")
    
    spatial_transform.randomize_parameters()
    input_spat = [spatial_transform(image) for image in input_temp]
    
    #for image in input_spat:
    #    print(type(image))
    #    image_np = image.numpy()
    #    im = PILIm.fromarray(image_np.astype('uint8'), 'RGB')
    #    im.save("croppedimage.png")
    
    im_dim = input_spat[0].size()[-2:]
    clip = torch.cat(input_spat, 0).view((duration, -1) + im_dim).permute(1, 0, 2, 3)
    input = clip[None, :, :, :, :] #Add 5th Dimension; batchSize=1
    #print(input.shape)
    return input

In [2]:
#GET PREDICTIONS
def ClassifyOutput(out, classestxt):
    with open(classestxt) as f:
        classes = [line.strip() for line in f.readlines()]
    
    #_, index = torch.max(out, 1)
    percentage = torch.nn.functional.softmax(out, dim=1)[0]
    #print(classes[index[0]], percentage[index[0]].item())
    
    _, indices = torch.sort(out, descending=True)
    #print([(classes[idx], percentage[idx].item()) for idx in indices[0][:80]])
    
    return(classes, indices, percentage)
    
    #return(classes[indices[0][0]], classes[indices[0][1]])
    
    #return(classes[index[0]], classes[index[1]])

In [26]:
def Classify(framesTT, verbose=False):
    load()
    model = load_checkpoint('jester_resnext_101_RGB_32.pth')
    
    spatialt, temporalt = getTransforms()
    input = TransformData(spatialt, temporalt, framesTT, verbose).cuda()
    model.eval()
    model = model.cuda()
    out = model(input)

    return ClassifyOutput(out, 'jesterclasses.txt')
